In [39]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import requests
import random
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import folium

In [40]:
CLIENT_ID='ZOO51K0TRLO3V1BXETAEBMWHHIO2C0DXTW05UWSNGTBS5G0E'
CLIENT_SECRET='ODKWIW1VFAKDYQ1PQNNJQU1RQDCA2QMV0UNTHXJHPBCR422B'
VERSION='20181511'
LIMIT=30

In [43]:
address = '102 North End Ave,New york,NY'
geolocator = Nominatim()

In [42]:
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
location

Location(Loopy Doopy Rooftop Bar, 102, North End Avenue, Battery Park City, Manhattan, Manhattan Community Board 1, New York County, NYC, New York, 10282, USA, (40.7149555, -74.0153365, 0.0))

In [44]:
search_query = 'Italian'
radius = 500

In [45]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [46]:
results = requests.get(url)

In [53]:
results_json = results.json()


In [54]:
venues = results_json['response']['venues']
df_venues = json_normalize(venues)
df_venues.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1542630870
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,Conca Cucina Italian Restaurant,v-1542630870
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,Ecco,v-1542630870


In [57]:
colnames = ['name','categories']+[col for col in df_venues.columns if col.startswith('location')]+['id']
df_venues_updated = df_venues.loc[:,colnames]
    

In [81]:
df_venues_updated

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [101]:
def get_category(row):
    category_list =  row['categories']
    print(category_list)
    if len(category_list)==0:
        return None
    else:
        return category_list[0]['name']

In [102]:
df_venues_updated.apply(get_category,axis=1)

[{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}]
[{'id': '4d4b7105d754a06374d81259', 'name': 'Food', 'pluralName': 'Food', 'shortName': 'Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]
[{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}]


0           Pizza Place
1                  Food
2    Italian Restaurant
dtype: object

In [103]:
df_venues_updated['categories'] = df_venues_updated.apply(get_category,axis=1)

[{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}]
[{'id': '4d4b7105d754a06374d81259', 'name': 'Food', 'pluralName': 'Food', 'shortName': 'Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]
[{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}]


In [104]:
df_venues_updated

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [112]:
df_venues_updated.columns = [col.split('.')[-1] for col in df_venues_updated.columns]

In [113]:
df_venues_updated.columns

Index(['name', 'categories', 'address', 'cc', 'city', 'country', 'crossStreet',
       'distance', 'formattedAddress', 'labeledLatLngs', 'lat', 'lng',
       'postalCode', 'state', 'id'],
      dtype='object')

In [114]:
df_venues_updated

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [161]:
venues_map = folium.Map(location=(latitude,longitude),zoom_start=13)

In [164]:
folium.features.CircleMarker(
[latitude,longitude],
radius=10,
color='red',
fill = True,
fill_color = 'red',
fill_opacity = 0.6,
popup='Conrad Hotel').add_to(venues_map)

In [154]:
df_venues_updated.columns

Index(['name', 'categories', 'address', 'cc', 'city', 'country', 'crossStreet',
       'distance', 'formattedAddress', 'labeledLatLngs', 'lat', 'lng',
       'postalCode', 'state', 'id'],
      dtype='object')

In [155]:
for name,lat, lng in zip(df_venues_updated['name'],df_venues_updated['lat'],df_venues_updated['lng']):
    print(name,lat,lng)

Harry's Italian Pizza Bar 40.71521779064671 -74.01473940209351
Conca Cucina Italian Restaurant 40.71446 -74.010086
Ecco 40.71533713859952 -74.00884766217825


In [162]:
for label,lat, lng in zip(df_venues_updated['categories'],df_venues_updated['lat'],df_venues_updated['lng']):
    folium.features.CircleMarker(
    [lat,lng],
    radius=5,
    color='blue',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6,
    popup=label).add_to(venues_map)

In [165]:
venues_map

In [176]:
VENUE_ID = '3fd66200f964a520f4e41ee3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(VENUE_ID,CLIENT_ID, CLIENT_SECRET,VERSION)

In [177]:
result = requests.get(url).json()

In [178]:
result['response']['venue'].keys() # drill down one by one starting with result.keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])

In [179]:
result['response']['venue']['rating']

7.5

In [184]:
result['response']['venue'].keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])

In [196]:
tips = result['response']['venue']['tips']['groups'][0]['items']

In [200]:
pd.set_option('display.max.colwidth',-1)
tips_df = json_normalize(tips)

In [201]:
tips_df

,agreeCount,canonicalUrl,createdAt,disagreeCount,entities,id,lang,likes.count,likes.groups,likes.summary,logView,text,todo.count,type,user.firstName,user.gender,user.id,user.lastName,user.photo.prefix,user.photo.suffix
0,2,https://foursquare.com/item/4c7ac41f278eb713ae245c80,1283114015,0,"[{'indices': [150, 187], 'type': 'url', 'object': {'url': 'http://www.downtownny.com/mapsguides/'}}]",4c7ac41f278eb713ae245c80,en,2,"[{'type': 'others', 'count': 2, 'items': [{'id': '9386209', 'firstName': 'The Official Khalis', 'gender': 'male', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/9386209-XJKHDWP30AUDGT4H.jpg'}}]}]",2 likes,True,"Old world comfort where you can enjoy veal chop alla salvia, lobster fra diavolo and branzino al forno. You can get our free Downtown dining guide at http://www.downtownny.com/mapsguides/",5,user,Downtown,male,2901547,Alliance,https://fastly.4sqi.net/img/user/,/TQWAQFUYL5LWNK2M.jpg


In [202]:
filtered_cols = ['text','agreeCount','disagreeCount','user.firstName','user.lastName','user.id']

In [204]:
tips_df_updated = tips_df.loc[:,filtered_cols]

In [205]:
tips_df_updated

,text,agreeCount,disagreeCount,user.firstName,user.lastName,user.id
0,"Old world comfort where you can enjoy veal chop alla salvia, lobster fra diavolo and branzino al forno. You can get our free Downtown dining guide at http://www.downtownny.com/mapsguides/",2,0,Downtown,Alliance,2901547


In [206]:
USER_ID='2901547'
url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(USER_ID,CLIENT_ID, CLIENT_SECRET,VERSION)

In [209]:
results = requests.get(url).json()

In [210]:
results.keys()

dict_keys(['meta', 'notifications', 'response'])

In [211]:
results['response'].keys()

dict_keys(['user'])

In [212]:
results['response']['user'].keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [213]:
results['response']['user']['tips']

{'count': 49}

In [215]:
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(USER_ID,CLIENT_ID, CLIENT_SECRET,VERSION)
results = requests.get(url).json()

In [216]:
results.keys()

dict_keys(['meta', 'response'])

In [217]:
results['response'].keys()

dict_keys(['tips'])

In [219]:
results['response']['tips'].keys()

dict_keys(['count', 'items'])

In [226]:
tips = json_normalize(results['response']['tips']['items'])

In [227]:
tips

,agreeCount,canonicalUrl,createdAt,disagreeCount,entities,id,lastUpvoteTimestamp,lastVoteText,likes.count,likes.groups,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,5,https://foursquare.com/item/4c818d3a51ada1cdab3b1410,1283558714,0,NaN,4c818d3a51ada1cdab3b1410,NaN,NaN,5,"[{'type': 'others', 'count': 5, 'items': [{'id': '69748385', 'firstName': 'Bo', 'lastName': 'Beyer-Johnson', 'gender': 'male', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/69748385-B5AFTRZSAHYRWLGS.jpg'}}, {'id': '59590054', 'firstName': 'Merkur', 'lastName': 'Maclang', 'gender': 'male', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/S5OCCAY1S22MCI3L.jpg'}}, {'id': '8344638', 'firstName': 'GiGi', 'gender': 'female', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/G2GAFDNYKI4ZCH5J.jpg'}}, {'id': '32365792', 'firstName': 'Tamika', 'lastName': '(Battle) Parker', 'gender': 'female', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/MZQZIACFBK0W40T2.jpg'}}]}]",...,"[{'label': 'display', 'lat': 40.704935, 'lng': -74.009786}]",40.704935,-74.009786,NaN,10004,NY,Leo's Bagels,0.0,"[{'type': 'venue', 'name': 'Venue photos', 'count': 0, 'items': [{'id': '50422eace4b03798679b5866', 'createdAt': 1346514604, 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/pJPUPhddWzH4iNbUDRI-1l7YO7SgW1jPVxrcOSgZ3F4.jpg', 'width': 540, 'height': 720, 'user': {'id': '14715054', 'firstName': 'Kathy', 'lastName': 'Elliott', 'gender': 'female', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/S0SFPNTZ0R5FTR1Y.jpg'}}, 'visibility': 'public'}, {'id': '4fae6a7ce4b091b462b09f02', 'createdAt': 1336830588, 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/1L8ECW92M92XmkiXxkwkKJP65K4yXc3cLd0a1Z5Vpyk.jpg', 'width': 540, 'height': 720, 'user': {'id': '19820', 'firstName': 'Lee', 'lastName': 'Hoffman', 'gender': 'male', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/RCHCZGFSNDFEG4OO.jpg'}}, 'visibility': 'public'}, {'id': '51c9a14a498e734d10ab2870', 'createdAt': 1372168522, 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/18051428_wb1a8e96VTFm1GePtIjFNHPsgRh9522REgtPlPJ6ZLI.jpg', 'width': 960, 'height': 720, 'user': {'id': '18051428', 'firstName': 'Dianna', 'lastName': 'H.', 'gender': 'female', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/AWPUKI3U5JYYUUQY.jpg'}}, 'visibility': 'public'}]}]",39688170
1,0,https://foursquare.com/item/4c7af2ff278eb7136fe05d80,1283126015,0,"[{'indices': [160, 197], 'type': 'url', 'object': {'url': 'http://www.downtownny.com/mapsguides/'}}]",4c7af2ff278eb7136fe05d80,NaN,NaN,4,"[{'type': 'others', 'count': 4, 'items': []}]",...,"[{'label': 'display', 'lat': 40.707491057618356, 'lng': -74.00860642537373}]",40.707491,-74.008606,NaN,10005,NY,William's Restaurant and Bar,NaN,NaN,NaN
2,0,https://foursquare.com/item/4c7af225278eb713e2db5d80,1283125797,0,"[{'indices': [148, 185], 'type': 'url', 'object': {'url': 'http://www.downtownny.com/mapsguides/'}}]",4c7af225278eb713e2db5d80,NaN,NaN,9,"[{'type': 'others', 'count': 9, 'items': []}]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,https://foursquare.com/item/4c7af1f5278eb71314db5d80,1283125749,0,"[{'indices': [127, 164], 'type': 'url', 'object': {'url': 'http://www.downtownny.com/mapsguides/'}}]",4c7af1f5278eb71314db5d80,NaN,NaN,0,[],...,"[{'label': 'display', 'lat': 40.70574392709303, 'lng': -74.01782141879579}]",40.705744,-74.017821,NaN,10004,NY,2 West At The Ritz Carlton,0.0,"[{'type': 'venue', 'name': 'Venue photos', 'count': 0, 'items': [{'id': '5b37bf19237dee002c0da855', 'createdAt': 1530380057, 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/13861948_fUT7tS3FeeAPQRE6XAI_2OyP-mZhKHAuYGqDXzEKviU.jpg', 'width': 1440, 'height': 1920, 'user': {'id': '13861948',